In [ ]:
import os
import numpy as np
import datetime
import math
import networkx as nx
import multiprocessing as mp
import random
from cdlib import algorithms


import time
import sys
import json

In [ ]:
var_dict_main = {}


In [ ]:
def local_path(adj, parameter=0.05):
    G = nx.Graph(adj)
    adj2 = nx.linalg.graphmatrix.adjacency_matrix(G).todense()
    #adj2 = nx.to_numpy_matrix(G)
    common = np.zeros(shape=(len(adj), len(adj)))
    common1 = np.zeros(shape=(len(adj), len(adj)))
    common2 = np.zeros(shape=(len(adj), len(adj)))
    adj2 = np.zeros(shape=(len(adj), len(adj)))
    for (u, v) in G.edges:
        adj2[u][v] = 1
        adj2[v][u] = 1
    print(len(G.edges))
    print("before lp matmul1")
    common1 = np.matmul(adj2, adj2)
    print("before lp matmul2")
    common2 = np.matmul(common1, adj2)
    common2 = common2 * parameter
    for i in range(len(adj)):
        for j in range(len(adj)):
            common[i][j] = common1[i][j] + common2[i][j]
    return common

In [ ]:
def gen_rand_edges_pool_single(num):
    starttime = time.time()
    G = var_dict_main['graph']
    print("inside pool num = "+str(num))
    seen = set()
    x, y = random.choice(list(G.nodes)), random.choice(list(G.nodes))
    t = 0
    while t < num:
        if not G.has_edge(x, y) and (y, x) not in seen:
            seen.add((x, y))
            t = t + 1
            #print(t)
        x, y = random.choice(list(G.nodes)), random.choice(list(G.nodes))

    print("after random edge generation inside pool "+str(len(seen)))
    endtime = time.time()
    currentDT = datetime.datetime.now()
    print(str(currentDT))
    #print(len(seen_inside))
    #print(seen_inside)

    return seen

In [ ]:
def gen_rand_edges(num, G):
    #print(str(G.nodes))
    #print(str(G.edges))
    num = num*5
    print("generating random edges")
    print(type(G))
    print("nodes = "+str(len(G.nodes)))
    print("edges = "+str(len(G.edges)))
    cpu = mp.cpu_count()
    cpu = cpu - 1
    print("cpu cores = " + str(cpu))
    arglist = []
    for i in range(cpu):
        arglist.append(int(num/cpu))
    var_dict_main['graph'] = G
    seen = set()
    with mp.Pool(cpu) as p:
        element = p.map(gen_rand_edges_pool_single,arglist)
        for e in element:
            seen.update(e)
    x, y = random.choice(list(G.nodes)), random.choice(list(G.nodes))
    t = len(seen)
    print("outside pool t = "+str(t)+" num = "+str(num))
    #print(seen)
    #time.sleep(100)
    while t < num:
        if not G.has_edge(x,y) and (y, x) not in seen:
            seen.add((x, y))
            t = t + 1
        x, y = random.choice(list(G.nodes)), random.choice(list(G.nodes))
    seen = list(seen)
    print("after random edge generation")
    return seen

In [ ]:
def data(m, t):
    print("reading dataset from file")
    data = open('./datasets_dynamic/'+str(t) + ".txt")
    edgelist = map(lambda q: list(map(int, q.split())), data.read().split("\n")[:-1])
    data.close()
    maxi = 0
    mini = 100000000000000000
    edgelist = list(edgelist)
    for x in edgelist:
        if x[-1] > maxi:
            maxi = x[-1]
        if x[-1] < mini:
            mini = x[-1]
    min1 = mini
    w = int((maxi - mini) / m)
    edgelist.sort(key=lambda x: x[-1])
    arr = []
    i = 0
    for i in range(0, m + 1):
        arr = arr + [min1 + w * i]
    arri = []
    # print(arr)
    nodes = set()
    for i in range(0, m):
        temp = []
        for j in edgelist:
            if j[-1] >= arr[i] and j[-1] <= arr[i + 1] and j[0]!=j[1]:
                temp += [[j[0], j[1]]]
        if temp != []:
            arri += [temp]
    # print(arri)
    # for x in arri:
    #     print(len(x))
    print("after read")
    #random.shuffle(arri)
    #print(str(arri))
    for item in arri: print(len(item))
    return arri

In [ ]:
def gen_graph(l):
    print("inside gen graph")
    t_graph = []
    node_set = set()
    max = -99999
    min = 99999
    for i in l:
        for edge in i:
            node_set.add(edge[0])
            node_set.add(edge[1])
            u = edge[0]
            v = edge[1]
            if u<v:
                if min > u:
                   min = u
                if max < v:
                    max = v
            else:
                if min > v:
                   min = v
                if max < u:
                    max = u
    print(str(min) + "-" + str(max))
    #sys.exit()
    edgelist_new = []
    count = -1
    for i in l:
        graph = nx.Graph()
        #graph.add_nodes_from(node_set)
        graph.add_edges_from(i)
        graph.remove_edges_from(nx.selfloop_edges(graph))
        t_graph.append(graph)
        edgelist_new.append(list(graph.edges))
    return [t_graph,edgelist_new]


In [ ]:
def partition_to_lp (x,y, graph, comm_list):
    count = 0
    comm_dict = dict()
    comm_no_indi = dict()
    #create dictionary which gives node communities
    for item in comm_list:
        count += 1
        comm_no_indi[count] = len(item)
        for node in item:
            comm_dict[node] = count
    #case to check faulty algorithm
    for node in graph:
        if node not in comm_dict.keys():
            print("non labelled node found")
            sys.exit()
    #getting 3 hop regions of x and y nodes
    reg_x = set()
    reg_y = set()
    reg_x.add(x)
    reg_y.add(y)
    #for 3 hop region
    for i in range(3):
        temp_x = set()
        temp_y = set()
        for curr in reg_x:
            temp_x.add(curr)
            for single in graph.neighbors(curr):
                temp_x.add(single)
        for curr in reg_y:
            temp_y.add(curr)
            for single in graph.neighbors(curr):
                temp_y.add(single)
        reg_x = temp_x
        reg_y = temp_y
    reg_x.remove(x)
    reg_y.remove(y)
    #intersection of ego regions
    common_reg = reg_x.intersection(reg_y)
    lp = 0
    count_comm = dict()
    #count number of instances of each community in region of common infleunce
    for key in comm_no_indi.keys():
        count_comm[key] = 0
    for item in common_reg:
        count_comm[comm_dict[item]] += 1
    for key in comm_no_indi.keys():
        if comm_dict[x] == comm_dict[y] == key:
            # special case for 3 equal labels to be defined
            #count_comm[key] = count_comm[key] / comm_no_indi[key]
            count_comm[key] = count_comm[key]
        else:
            count_comm[key] = count_comm[key]/comm_no_indi[key]
    for key in count_comm.keys():
        lp += count_comm[key]
    return lp


In [ ]:
def features_comm_opti(m, t, f_no=12):
    identity = " algo - comm dyn dataset - " + str(t)

    data_curr = data(m, t)
    l1 = data_curr
    l = []
    for i in l1:
        edgelist = list(set(tuple(sorted(sub)) for sub in i))
        l.append(edgelist)

    t_graph = gen_graph(l)[0]
    l = gen_graph(l)[1]

    teCt = l[m - 1]
    reCt = gen_rand_edges(len(teCt), t_graph[m - 1])

    for i in reCt:
        teCt.append(i)

    teCt = list(set(tuple(sorted(sub)) for sub in teCt))

    list1 = []
    print("before graph" + str(identity))
    edges_dict = dict()
    for t1 in range(0, m - 1):
        starttime = time.time()
        print("before dictionary" + str(identity))
        sum = 0
        count = 0
        dict_count = 0
        dict_node = dict()
        G = t_graph[t1]
        print(str(G))
        for node in G.nodes:
            dict_node[node] = dict_count
            dict_count += 1
        currentDT = datetime.datetime.now()
        print(str(currentDT))
        from cdlib.algorithms import cpm, label_propagation,\
            der, eigenvector, sbm_dl_nested, walktrap, surprise_communities, \
            spinglass, significance_communities, rb_pots, rber_pots, pycombo, \
            paris, mcode, markov_clustering, lswl_plus, infomap, greedy_modularity, \
            leiden

        current_comm = greedy_modularity(G)
        print(type(current_comm))
        current_comm_dict = json.loads(current_comm.to_json())
        current_comm_list_greed = current_comm_dict['communities']
        print("no. of communities greed - " + str(len(current_comm_list_greed)))
        for item in current_comm_list_greed:
            print(item)
        print("after community greed")

        # ... (rest of the code remains unchanged)

    # Print contents of teCt for debugging
    print("teCt:", teCt)

    for e in teCt:
        edge_key = str(e[0]) + "+" + str(e[1])
        if edge_key in edges_dict:
            list1.append(list(edges_dict[edge_key]))
        else:
            print(f"Edge {e} not found in edges_dict")
            # Handle the missing edge if needed

    # ... (rest of your code)

    f = np.array(list1)
    np.take(f, np.random.permutation(f.shape[0]), axis=0, out=f)
    # print("list1 = " + str(list1))
    print("after get feature" + str(identity))
    print("shape of feature = " + str(f.shape))
    return f


In [ ]:
''''
def features_comm_opti(m, t, f_no=12):
    identity = " algo - comm dyn dataset - " + str(t)

    data_curr = data(m, t)
    l1 = data_curr
    l = []
    for i in l1:
        edgelist = list(set(tuple(sorted(sub)) for sub in i))
        l.append(edgelist)

    t_graph = gen_graph(l)[0]
    l = gen_graph(l)[1]

    teCt = l[m - 1]
    reCt = gen_rand_edges(len(teCt), t_graph[m - 1])

    for i in reCt:
        teCt.append(i)

    teCt = list(set(tuple(sorted(sub)) for sub in teCt))

    list1 = []
    print("before graph" + str(identity))
    edges_dict = dict()
    for t1 in range(0, m - 1):
        starttime = time.time()
        print("before dictionary" + str(identity))
        sum = 0
        count = 0
        dict_count = 0
        dict_node = dict()
        G = t_graph[t1]
        print(str(G))
        for node in G.nodes:
            dict_node[node] = dict_count
            dict_count += 1
        currentDT = datetime.datetime.now()
        print(str(currentDT))
        from cdlib.algorithms import cpm, label_propagation,\
            der, eigenvector, sbm_dl_nested, walktrap, surprise_communities, \
            spinglass, significance_communities, rb_pots, rber_pots, pycombo, \
            paris, mcode, markov_clustering, lswl_plus, infomap, greedy_modularity, \
            leiden

        current_comm = greedy_modularity(G)
        print(type(current_comm))
        current_comm_dict = json.loads(current_comm.to_json())
        current_comm_list_greed = current_comm_dict['communities']
        print("no. of communities greed - " + str(len(current_comm_list_greed)))
        for item in current_comm_list_greed:
            print(item)
        print("after community greed")

        # ... (rest of the code remains unchanged)

    for e in teCt:
        edge_key = str(e[0]) + "+" + str(e[1])
        list1.append(list(edges_dict[edge_key]))
    f = np.array(list1)
    np.take(f, np.random.permutation(f.shape[0]), axis=0, out=f)
    # print("list1 = " + str(list1))
    print("after get feature" + str(identity))
    print("shape of feature = " + str(f.shape))
    return f
'''

In [ ]:
m = 5
t = "mit"
data_curr = data(m,t)

reading dataset from file
after read
315620
387886
114573
166764
101561


In [ ]:
graph_info = gen_graph(data_curr)
t_graph = graph_info[0]
edgelist_new = graph_info[1]

inside gen graph
1-96


In [ ]:
num_edges_to_generate = 362*2
random_edges = gen_rand_edges(num_edges_to_generate, t_graph[m - 1])

generating random edges
<class 'networkx.classes.graph.Graph'>
nodes = 60
edges = 362
cpu cores = 1
inside pool num = 3620
after random edge generation inside pool 1441
2024-01-29 18:18:54.426580
outside pool t = 1441 num = 3620
after random edge generation


In [ ]:
random_edges_pool = gen_rand_edges_pool_single(120)

inside pool num = 120
after random edge generation inside pool 116
2024-01-29 18:21:26.330596


In [ ]:
adj_matrix = np.random.rand(60, 60)
common_path_matrix = local_path(adj_matrix)

1830
before lp matmul1
before lp matmul2


In [ ]:
from cdlib.algorithms import greedy_modularity

In [ ]:
# Example: Using communities detected by greedy_modularity algorithm
current_comm_greed = greedy_modularity(t_graph[m - 1])
current_comm_dict_greed = json.loads(current_comm_greed.to_json())
current_comm_list_greed = current_comm_dict_greed['communities']

# Example call to partition_to_lp
x, y = 1, 2  # Replace with your actual node values
lp_score = partition_to_lp(x, y, t_graph[m - 1], current_comm_list_greed)


In [ ]:
print(lp_score)

3.8968253968253967


In [ ]:
feature_matrix = features_comm_opti(m, t, f_no=12)

reading dataset from file
after read
315620
387886
114573
166764
101561
inside gen graph
1-96
inside gen graph
1-96
generating random edges
<class 'networkx.classes.graph.Graph'>
nodes = 60
edges = 362
cpu cores = 1
inside pool num = 1810
after random edge generation inside pool 1192
2024-01-29 18:42:44.814620
outside pool t = 1192 num = 1810
after random edge generation
before graph algo - comm dyn dataset - mit
before dictionary algo - comm dyn dataset - mit
Graph with 95 nodes and 1792 edges
2024-01-29 18:42:44.875001
<class 'cdlib.classes.node_clustering.NodeClustering'>
no. of communities greed - 2
[1, 2, 3, 4, 5, 6, 8, 9, 11, 12, 13, 14, 16, 18, 19, 21, 25, 28, 29, 31, 35, 38, 41, 42, 43, 44, 45, 46, 47, 48, 49, 58, 60, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 91, 93, 94, 95]
[7, 10, 15, 17, 20, 22, 23, 24, 26, 27, 30, 32, 33, 34, 36, 37, 39, 40, 50, 51, 52, 53, 54, 55, 56, 57, 59, 61, 62, 63, 67, 79, 87, 88, 89, 90, 92]
after community 

In [ ]:
print(feature_matrix)

NameError: name 'feature_matrix' is not defined